## Schedule Table Crawling

**September**

In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True
driver = webdriver.Chrome('/Applications/chromedriver', options=options)

In [2]:
driver.get('https://www.koreabaseball.com/Schedule/Schedule.aspx')
driver.find_element_by_id('ddlMonth').send_keys('09')

table = driver.find_element_by_xpath('//*[@id="tblSchedule"]/tbody')
schedule = table.text

In [3]:
a = []

In [4]:
for i in range(len(schedule.split('\n'))):
    if len(schedule.split('\n')[i].split(' ')) == 5:
        a.append(i)

In [5]:
b = []

In [6]:
for i in range(len(schedule.split('\n'))):
    b.append(schedule.split('\n')[i].split(' '))

In [7]:
for i in a:
    del b[i][0]

In [8]:
schedule_sep = pd.DataFrame(b).drop([3], axis=1)
schedule_sep.columns = ['time', 'game', 'stadium']
schedule_sep['date'] = None
schedule_sep = schedule_sep[['date', 'time', 'game', 'stadium']]

In [9]:
a.append(len(schedule.split('\n')))

In [10]:
for i in range(len(a)-1):
    schedule_sep.iloc[a[i]:a[i+1], 0] = schedule.split('\n')[a[i]].split(' ')[0]

In [11]:
schedule_sep

,date,time,game,stadium
0,09.01(화),18:30,한화vs두산,잠실
1,09.01(화),18:30,LGvsSK,문학
2,09.01(화),18:30,롯데vsKT,수원
3,09.01(화),18:30,삼성vsKIA,광주
4,09.01(화),18:30,NCvs키움,고척
...,...,...,...,...
130,09.30(수),14:00,롯데vsLG,잠실
131,09.30(수),14:00,KTvs삼성,대구
132,09.30(수),14:00,SKvsNC,창원
133,09.30(수),14:00,KIAvs키움,고척


**October**

In [12]:
driver.get('https://www.koreabaseball.com/Schedule/Schedule.aspx')
driver.find_element_by_id('ddlMonth').send_keys('10')

table = driver.find_element_by_xpath('//*[@id="tblSchedule"]/tbody')
schedule = table.text

In [13]:
a = []

In [14]:
for i in range(len(schedule.split('\n'))):
    if len(schedule.split('\n')[i].split(' ')) == 5:
        a.append(i)

In [15]:
b = []

In [16]:
for i in range(len(schedule.split('\n'))):
    b.append(schedule.split('\n')[i].split(' '))

In [17]:
for i in a:
    del b[i][0]

In [18]:
schedule_oct = pd.DataFrame(b).drop([3], axis=1)
schedule_oct.columns = ['time', 'game', 'stadium']
schedule_oct['date'] = None
schedule_oct = schedule_oct[['date', 'time', 'game', 'stadium']]

In [19]:
a.append(len(schedule.split('\n')))

In [20]:
for i in range(len(a)-1):
    schedule_oct.iloc[a[i]:a[i+1], 0] = schedule.split('\n')[a[i]].split(' ')[0]

In [21]:
schedule_oct

,date,time,game,stadium
0,10.01(목),14:00,롯데vsLG,잠실
1,10.01(목),14:00,KTvs삼성,대구
2,10.01(목),14:00,SKvsNC,창원
3,10.01(목),14:00,KIAvs키움,고척
4,10.01(목),14:00,두산vs한화,대전
...,...,...,...,...
79,10.18(일),14:00,KIAvsLG,잠실
80,10.18(일),14:00,KTvsSK,문학
81,10.18(일),14:00,롯데vsNC,창원
82,10.18(일),14:00,두산vs키움,고척


In [22]:
schedule = pd.concat([schedule_sep, schedule_oct])

In [23]:
schedule = schedule.iloc[125:,]
schedule = schedule.reset_index(drop=True)

In [24]:
schedule

,date,time,game,stadium
0,09.29(화),18:30,롯데vsLG,잠실
1,09.29(화),18:30,KTvs삼성,대구
2,09.29(화),18:30,SKvsNC,창원
3,09.29(화),18:30,KIAvs키움,고척
4,09.29(화),18:30,두산vs한화,대전
...,...,...,...,...
89,10.18(일),14:00,KIAvsLG,잠실
90,10.18(일),14:00,KTvsSK,문학
91,10.18(일),14:00,롯데vsNC,창원
92,10.18(일),14:00,두산vs키움,고척


**Tansform**

In [25]:
schedule['T'] = None
schedule['B'] = None

In [26]:
for i in range(schedule.shape[0]):
    schedule.iloc[i, 4] = schedule.iloc[i, 2].split('vs')[0]
    schedule.iloc[i, 5] = schedule.iloc[i, 2].split('vs')[1]

In [27]:
for i in range(schedule.shape[0]):
    schedule.iloc[i, 0] = '2020.' + schedule.iloc[i, 0][:5]
    schedule.iloc[i, 0] = schedule.iloc[i, 0].replace('.', '')

In [28]:
name = {'NC':'NC', '두산':'OB', '키움':'WO', 'KIA':'HT', 'LG':'LG',
       '삼성':'SS', 'KT':'KT', '롯데':'LT', 'SK':'SK', '한화':'HH'}

f = lambda x: name.get(x, x)
schedule['T'] = schedule['T'].map(f)
schedule['B'] = schedule['B'].map(f)

In [29]:
schedule = pd.DataFrame(schedule['date'] + schedule['T'] + schedule['B'], columns=['G_ID'])

In [30]:
for i in schedule[schedule.duplicated(['G_ID'], keep=False) == True].index:
    schedule.iloc[i, 0] = schedule.iloc[i, 0] + '1'
for i in schedule[schedule.duplicated(['G_ID'], keep=False) == False].index:
    schedule.iloc[i, 0] = schedule.iloc[i, 0] + '0'

In [31]:
schedule['TEAM'] = None
schedule['TB_SC'] = None

In [32]:
schedule_B = schedule.copy()

In [33]:
for i in range(schedule.shape[0]):
    schedule.iloc[i, 1] = schedule.iloc[i, 0][8:10]
    schedule.iloc[i, 2] = 'T'
for i in range(schedule_B.shape[0]):
    schedule_B.iloc[i, 1] = schedule_B.iloc[i, 0][10:12]
    schedule_B.iloc[i, 2] = 'B'

In [34]:
schedule = pd.concat([schedule, schedule_B]).reset_index(drop=True)

In [35]:
schedule

,G_ID,TEAM,TB_SC
0,20200929LTLG0,LT,T
1,20200929KTSS0,KT,T
2,20200929SKNC0,SK,T
3,20200929HTWO0,HT,T
4,20200929OBHH0,OB,T
...,...,...,...
183,20201018HTLG0,LG,B
184,20201018KTSK0,SK,B
185,20201018LTNC0,NC,B
186,20201018OBWO0,WO,B


In [36]:
schedule.to_csv('schedule.csv', index=False)